In [1]:
import pandas as pd
import numpy as np
from functools import reduce
from IPython.display import display
import re

In [2]:
vaccinationsDF = pd.read_csv('dataFiles/vaccinations.csv', sep=',', header=0)
casesDF = pd.read_csv('dataFiles/covidCases.csv', sep=',', header=0)
iso_codesDF = pd.read_csv('dataFiles/iso_codes.csv', sep=',', header=1)

In [3]:
casesDF.rename(columns={'Date_reported':'date', 'Country':'location'}, inplace=True)
iso_codesDF.drop(iso_codesDF.columns[[1, 2, 5, 6, 7]], axis=1, inplace=True)
iso_codesDF.rename(columns={'Country name':'location', 'Alpha-2 code':'iso2Code', 'Alpha-3 code':'iso3Code'}, inplace=True)


In [4]:
iso_codesDF = iso_codesDF.astype(str)
iso_codesDF.drop(iso_codesDF[(iso_codesDF['iso2Code'].apply(len) != 2) | (iso_codesDF['iso3Code'].apply(len) != 3)].index, inplace=True)
# iso_codesDF = iso_codesDF['iso2Code'].apply(lambda x: x if len(x) == 2 else iso_codesDF[x.index].drop(axis=1))

# iso_codesDF = iso_codesDF.loc[iso_codesDF.iloc[:, 1].str.contains(r'?![a-zA-Z]{2}', regex=True, na=False, flags=re.IGNORECASE)]
    # iso_codesDF.iso2Code == r'^[a-zA-Z]{2}') | (iso_codesDF.iso3Code == r'^[a-zA-Z]{3}')]
display(iso_codesDF.count())

#display(iso_codesDF.head(5))

location    248
iso2Code    248
iso3Code    248
dtype: int64

In [ ]:
display(casesDF.isnull().sum())

display(casesDF['Country'].where(casesDF['Country_code'].isnull()).unique())
display(vaccinationsDF.isnull().sum())

In [9]:
display(len(casesDF.location.unique()))
display(len(vaccinationsDF.location.unique()))
display(len(iso_codesDF.location.unique()))
# display(casesDF.tail(5))
# display(vaccinationsDF.tail(5))

# display(vaccinationsDF)

237

235

248

In [10]:
def common_member(a, b):
    result = [i for i in a if i in b]
    return len(result)

display(common_member(casesDF.location.unique(), vaccinationsDF.location.unique()))

display(common_member(casesDF.Country_code.unique(), iso_codesDF.iso2Code.unique()))
display(common_member(vaccinationsDF.iso_code.unique(), iso_codesDF.iso3Code.unique()))

189

231

216

In [25]:
df = reduce(lambda left, right: pd.merge(left, right, on=['location', 'date'], how='outer'), [casesDF, vaccinationsDF])
df = df.sort_values(by=['date', 'location'])
display(df.head(5))
df.to_csv('merged.csv', header=True, sep=';')

,date,Country_code,location,WHO_region,New_cases,Cumulative_cases,New_deaths,Cumulative_deaths,iso_code,total_vaccinations,...,total_boosters,daily_vaccinations_raw,daily_vaccinations,total_vaccinations_per_hundred,people_vaccinated_per_hundred,people_fully_vaccinated_per_hundred,total_boosters_per_hundred,daily_vaccinations_per_million,daily_people_vaccinated,daily_people_vaccinated_per_hundred
232023,719163,NaN,Afghanistan,NaN,NaN,NaN,NaN,NaN,AFG,0.0,...,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN
232024,719163,NaN,Afghanistan,NaN,NaN,NaN,NaN,NaN,AFG,NaN,...,NaN,NaN,1367.0,NaN,NaN,NaN,NaN,34.0,1367.0,0.003
232025,719163,NaN,Afghanistan,NaN,NaN,NaN,NaN,NaN,AFG,NaN,...,NaN,NaN,1367.0,NaN,NaN,NaN,NaN,34.0,1367.0,0.003
232026,719163,NaN,Afghanistan,NaN,NaN,NaN,NaN,NaN,AFG,NaN,...,NaN,NaN,1367.0,NaN,NaN,NaN,NaN,34.0,1367.0,0.003
232027,719163,NaN,Afghanistan,NaN,NaN,NaN,NaN,NaN,AFG,NaN,...,NaN,NaN,1367.0,NaN,NaN,NaN,NaN,34.0,1367.0,0.003


In [26]:
display(df.columns)

Index(['date', 'Country_code', 'location', 'WHO_region', 'New_cases',
       'Cumulative_cases', 'New_deaths', 'Cumulative_deaths', 'iso_code',
       'total_vaccinations', 'people_vaccinated', 'people_fully_vaccinated',
       'total_boosters', 'daily_vaccinations_raw', 'daily_vaccinations',
       'total_vaccinations_per_hundred', 'people_vaccinated_per_hundred',
       'people_fully_vaccinated_per_hundred', 'total_boosters_per_hundred',
       'daily_vaccinations_per_million', 'daily_people_vaccinated',
       'daily_people_vaccinated_per_hundred'],
      dtype='object')

In [23]:
display(df.count())

date                                   357647
Country_code                           231044
location                               357647
WHO_region                             232023
New_cases                              232023
Cumulative_cases                       232023
New_deaths                             232023
Cumulative_deaths                      232023
iso_code                               125624
total_vaccinations                      62397
people_vaccinated                       59634
people_fully_vaccinated                 56968
total_boosters                          32720
daily_vaccinations_raw                  51733
daily_vaccinations                     124919
total_vaccinations_per_hundred          62397
people_vaccinated_per_hundred           59634
people_fully_vaccinated_per_hundred     56968
total_boosters_per_hundred              32720
daily_vaccinations_per_million         124919
daily_people_vaccinated                124175
daily_people_vaccinated_per_hundre

In [ ]:
vaccinationsDF['date'] = pd.to_datetime(vaccinationsDF['date'], format='%Y-%m-%d').apply(lambda x: x.toordinal())
casesDF['Date_reported'] = pd.to_datetime(casesDF['Date_reported'], format='%Y-%m-%d').apply(lambda x: x.toordinal())